# Import Libraris

In [15]:
!pip install evidently

In [1]:
pip install ydata_profiling

  Using cached pandas-2.3.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached statsmodels-0.14.5-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.5 kB)
  Using cached pillow-11.3.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.0 kB)
  Using cached contourpy-1.3.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.59.0-cp312-cp312-macosx_10_13_universal2.whl.metadata (107 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.2 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
  Using cached pytz-2025.2-py2.py3-none-a

In [9]:
pip install facets-overview

Note: you may need to restart the kernel to use updated packages.


In [18]:
import pandas as pd

from IPython.core.display import HTML, display_html
import base64
from facets_overview.generic_feature_statistics_generator import GenericFeatureStatisticsGenerator

from ydata_profiling import ProfileReport

from evidently import Dataset, DataDefinition, Report
from evidently.presets import DataDriftPreset, DataSummaryPreset

# Load data and EDA

In [6]:
df = pd.read_csv(''
                 'Hotel Reservations.csv')

In [7]:
profile = ProfileReport(df, title="Hotel Bookings Profiling", minimal=True)

profile.to_file("report.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 185.81it/s]


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Booking_ID                            36275 non-null  object 
 1   no_of_adults                          36275 non-null  int64  
 2   no_of_children                        36275 non-null  int64  
 3   no_of_weekend_nights                  36275 non-null  int64  
 4   no_of_week_nights                     36275 non-null  int64  
 5   type_of_meal_plan                     36275 non-null  object 
 6   required_car_parking_space            36275 non-null  int64  
 7   room_type_reserved                    36275 non-null  object 
 8   lead_time                             36275 non-null  int64  
 9   arrival_year                          36275 non-null  int64  
 10  arrival_month                         36275 non-null  int64  
 11  arrival_date   

In [17]:
proto = GenericFeatureStatisticsGenerator().ProtoFromDataFrames([{'name': 'train', 'table': df}])

protostr = base64.b64encode(proto.SerializeToString()).decode("utf-8")
HTML_TEMPLATE = """
<script src="https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/1.3.3/webcomponents-lite.js"></script>
<link rel="import" href="https://raw.githubusercontent.com/PAIR-code/facets/master/facets-dist/facets-jupyter.html">
<facets-overview id="elem"></facets-overview>
<script>
document.querySelector("#elem").protoInput = "{protostr}";
</script>"""
html = HTML_TEMPLATE.format(protostr=protostr)
display_html(html)

/Users/domager73/PycharmProjects/Hoter_reser_harbour/.venv/lib/python3.12/site-packages/facets_overview/base_generic_feature_statistics_generator.py:121: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  flattened = x.ravel()


# Feature Genering

In [34]:
def target_to_int(df):
    df['booking_status'] = (df['booking_status'] == 'Canceled').astype('int64')
    return df

In [35]:
df['date'] = df.apply(
    lambda row: ''.join(map(str, [row['arrival_year'],row['arrival_month'], row['arrival_date']])), 
    axis=1
)

In [36]:
df = df.sort_values(by = 'date')

In [37]:
df = target_to_int(df)

In [42]:
df

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status,date
21771,INN21772,2,0,2,3,Meal Plan 1,0,Room_Type 1,6,2017,10,1,Offline,0,0,0,58.40,0,0,2017101
3866,INN03867,1,0,0,1,Meal Plan 1,0,Room_Type 1,1,2017,10,1,Offline,0,0,0,96.00,0,0,2017101
34227,INN34228,2,0,0,2,Meal Plan 1,0,Room_Type 1,16,2017,10,1,Online,0,0,0,117.00,2,0,2017101
34104,INN34105,1,0,0,2,Meal Plan 1,0,Room_Type 1,17,2017,10,1,Online,0,0,0,73.88,3,0,2017101
6144,INN06145,2,0,1,3,Meal Plan 1,0,Room_Type 1,16,2017,10,1,Online,0,0,0,111.75,2,0,2017101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13628,INN13629,2,0,1,2,Meal Plan 1,0,Room_Type 1,198,2018,9,9,Offline,0,0,0,90.00,0,1,201899
15109,INN15110,2,0,1,2,Meal Plan 1,0,Room_Type 1,198,2018,9,9,Offline,0,0,0,90.00,0,1,201899
31588,INN31589,2,0,0,2,Meal Plan 1,0,Room_Type 1,169,2018,9,9,Offline,0,0,0,80.75,0,0,201899
16462,INN16463,2,0,0,2,Meal Plan 1,0,Room_Type 4,65,2018,9,9,Online,0,0,0,149.40,1,1,201899


# Calculate historical data

In [55]:
T = 10000
his_df = df[:T]

In [56]:
P_his = his_df['booking_status'].sum() / T
P_his

0.2561

In [58]:
Avg_rev_his = his_df['avg_price_per_room'].sum() / T
Avg_rev_his

97.349209

In [60]:
Loss = P_his * T * Avg_rev_his
Loss

249311.324249